In [10]:
# Import the required libraries: Pandas, Numpy, Matplotlib and   Seaborn

import pandas as pd          
import numpy as np 
import seaborn as sns # For mathematical calculations
import matplotlib.pyplot as plt  # For plotting graphs
from datetime import datetime    # To access datetime
from pandas import Series # To work on series
%matplotlib inline
import warnings                   # To ignore the warnings
warnings.filterwarnings("ignore")

# set seed for reproducibility
np.random.seed(0)

In [12]:
# fuzz is used to compare TWO strings
from fuzzywuzzy import fuzz

# process is used to compare a string to MULTIPLE other strings
from fuzzywuzzy import process

import chardet


In [20]:
# Load and read the data 

df=pd.read_csv("PakistanSuicideAttacks Ver 6 (10-October-2017).csv",encoding = "ISO-8859-1")
df.head(3)

,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,...,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
0,1,Sunday-November 19-1995,25 Jumaada al-THaany 1416 A.H,Holiday,Weekend,NaN,Islamabad,33.7180,73.0718,Capital,...,None,14.0,15.0,NaN,60,2.0,NaN,NaN,15.835,60.503
1,2,Monday-November 6-2000,10 SHa`baan 1421 A.H,Working Day,NaN,NaN,Karachi,24.9918,66.9911,Sindh,...,None,NaN,3.0,NaN,3,1.0,NaN,NaN,23.770,74.786
2,3,Wednesday-May 8-2002,25 safar 1423 A.H,Working Day,NaN,7:45 AM,Karachi,24.9918,66.9911,Sindh,...,Christian,13.0,15.0,20.0,40,1.0,2.5 Kg,1.Jinnah Postgraduate Medical Center 2. Civil ...,31.460,88.628


Conduct preliminary text pre-processing

 Here  I'm interested in cleaning up the "" Column to make sure there's no data entry inconsistencies in it.
 we will go through and check each row by hand and correct inconsistencies where we find them


In [22]:
# Get all the unique values in the 'City' column

City= df["City"].unique()
City


array(['Islamabad', 'Karachi', 'Karachi ', 'Quetta', 'Rawalpindi',
       'North waziristan', 'Kohat', 'Attock ', 'Sialkot', 'Lahore ',
       'Swat', 'Hangu', 'Bannu', 'North Waziristan', 'Lasbela',
       'Malakand ', 'Peshawar', 'Peshawar ', 'D.I Khan ', 'Lakki Marwat',
       'Tank', 'Islamabad ', 'Quetta ', 'Gujrat', 'Charsadda', 'Tank ',
       'Swat ', 'D.I Khan', 'Kohat ', 'Kuram Agency ', 'Shangla ',
       'Bajaur Agency', 'South Waziristan', 'Haripur', 'bannu',
       'karachi ', 'Sargodha', 'Nowshehra', 'Charsadda ',
       'Mohmand agency', 'Dara Adam Khel', 'Khyber Agency', 'Mardan',
       'Bhakkar ', 'Orakzai Agency', 'Mohmand Agency', 'Buner',
       'D.G Khan ', 'Pishin', 'Chakwal ', 'Lahore', 'Upper Dir',
       'Muzaffarabad', 'swat', 'Totalai', 'Multan', 'D.G Khan',
       'Lower Dir', 'Lakki marwat', 'Sudhanoti', 'Khyber Agency ',
       'peshawar', 'Poonch', 'Mansehra', 'Karak', 'Mohmand Agency ',
       'South waziristan', 'lakki marwat', 'Swabi ', 'Dara Adam kh

In [59]:
# Sort the City column rows alphabetically and then take a closer look

City.sort()
City

array(['ATTOCK', 'Attock ', 'Bajaur Agency', 'Bannu', 'Bhakkar ', 'Buner',
       'Chakwal ', 'Chaman', 'Charsadda', 'Charsadda ', 'D. I Khan',
       'D.G Khan', 'D.G Khan ', 'D.I Khan', 'D.I Khan ', 'Dara Adam Khel',
       'Dara Adam khel', 'Fateh Jang', 'Ghallanai, Mohmand Agency ',
       'Gujrat', 'Hangu', 'Haripur', 'Hayatabad', 'Islamabad',
       'Islamabad ', 'Jacobabad', 'KURRAM AGENCY', 'Karachi', 'Karachi ',
       'Karak', 'Khanewal', 'Khuzdar', 'Khyber Agency', 'Khyber Agency ',
       'Kohat', 'Kohat ', 'Kuram Agency ', 'Lahore', 'Lahore ',
       'Lakki Marwat', 'Lakki marwat', 'Lasbela', 'Lower Dir', 'MULTAN',
       'Malakand ', 'Mansehra', 'Mardan', 'Mohmand Agency',
       'Mohmand Agency ', 'Mohmand agency', 'Mosal Kor, Mohmand Agency',
       'Multan', 'Muzaffarabad', 'North Waziristan', 'North waziristan',
       'Nowshehra', 'Orakzai Agency', 'Peshawar', 'Peshawar ', 'Pishin',
       'Poonch', 'Quetta', 'Quetta ', 'Rawalpindi', 'Sargodha',
       'Sehwan town',


There are problems due to inconsistent data entry: 'Lahore' and 'Lahore ', for example, or 'Lakki Marwat' and 'Lakki marwat'.

Fixing the text data entry inconsistencies

Make everything into lowercase in the dataset.

Remove the white spaces at the beginnign and the end of the cells.

This will remove the inconsistencies in capitalizations and trailing white spaces in our City Column text data.

In [81]:
# Convert the rows in the City Column to lower case from the dataset
# Set the values in the dataset df and column city to lower case


df["City"]= df['City'].str.lower()
df["City"].sample(4)

60     bajaur agency
83        rawalpindi
257            kohat
193    khyber agency
Name: City, dtype: object

In [82]:
# Remove the trailing white spaces in the City Column

df["City"] =df['City'].str.strip()
df["City"].sample(4)


92     north waziristan
210              attock
429            peshawar
20              karachi
Name: City, dtype: object

Lets tackle more difficult Inconsistencies

Use fuzzy matching to correct inconsistent data entry

Still in the City Column, lets have a lok and see if there is more data cleaning that we need to do

In [83]:
# Get all the unique values in the 'City' column

City = df['City'].unique()
City

array(['islamabad', 'karachi', 'quetta', 'rawalpindi', 'north waziristan',
       'kohat', 'attock', 'sialkot', 'lahore', 'swat', 'hangu', 'bannu',
       'lasbela', 'malakand', 'peshawar', 'd.i khan', 'lakki marwat',
       'tank', 'gujrat', 'charsadda', 'kuram agency', 'shangla',
       'bajaur agency', 'south waziristan', 'haripur', 'sargodha',
       'nowshehra', 'mohmand agency', 'dara adam khel', 'khyber agency',
       'mardan', 'bhakkar', 'orakzai agency', 'buner', 'd.g khan',
       'pishin', 'chakwal', 'upper dir', 'muzaffarabad', 'totalai',
       'multan', 'lower dir', 'sudhanoti', 'poonch', 'mansehra', 'karak',
       'swabi', 'shikarpur', 'sukkur', 'chaman', 'd. i khan', 'khanewal',
       'fateh jang', 'taftan', 'tirah valley', 'wagah', 'zhob',
       'kurram agency', 'taunsa', 'jacobabad', 'shabqadar-charsadda',
       'khuzdar', 'ghallanai, mohmand agency', 'hayatabad',
       'mosal kor, mohmand agency', 'sehwan town',
       'tangi, charsadda district'], dtype=object

In [85]:
# Sort the City 2 Alphabetically and then take a closer look

City.sort()
City

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd. i khan', 'd.g khan', 'd.i khan',
       'dara adam khel', 'fateh jang', 'ghallanai, mohmand agency',
       'gujrat', 'hangu', 'haripur', 'hayatabad', 'islamabad',
       'jacobabad', 'karachi', 'karak', 'khanewal', 'khuzdar',
       'khyber agency', 'kohat', 'kuram agency', 'kurram agency',
       'lahore', 'lakki marwat', 'lasbela', 'lower dir', 'malakand',
       'mansehra', 'mardan', 'mohmand agency',
       'mosal kor, mohmand agency', 'multan', 'muzaffarabad',
       'north waziristan', 'nowshehra', 'orakzai agency', 'peshawar',
       'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'],
      dtype=



There are  some remaining inconsistencies: 'd. i khan' and 'd.i khan' should probably be the same.
(I looked it up and 'd.g khan' is a seperate city, so I shouldn't combine those.)

 Using Fuzzywuzzy package to help identify which string are closest to each other.




Fuzzy matching: The process of automatically finding text strings that are very similar to the target string. In general, a string is considered "closer" to another one the fewer characters you'd need to change if you were transforming one string into another. So "apple" and "snapple" are two changes away from each other (add "s" and "n") while "in" and "on" and one change away (rplace "i" with "o"). You won't always be able to rely on fuzzy matching 100%, but it will usually end up saving you at least a little time.

Fuzzywuzzy returns a ratio given two strings. The closer the ratio is to 100, the smaller the edit distance between the two strings. Here, we're going to get the ten strings from our list of cities that have the closest distance to "d.i khan".

In [87]:
# Get the top 10 closest matches to "d.i khan"

Matches = fuzzywuzzy.process.extract("d.i khan", Cities, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
Matches

[('D.I Khan ', 100),
 ('D.I Khan', 100),
 ('D. I Khan', 100),
 ('D.G Khan ', 88),
 ('D.G Khan', 88),
 ('Khanewal', 50),
 ('Sudhanoti', 47),
 ('Kohat', 46),
 ('Hangu', 46),
 ('Kohat ', 46)]

We can see that two of the items in the cities are very close to "d.i khan": "d. i khan" and "d.i khan". We can also see the "d.g khan", which is a seperate city, has a ratio of 88. Since we don't want to replace "d.g khan" with "d.i khan", let's replace all rows in our City column that have a ratio of > 90 with "d. i khan".

To do this, I'm going to write a function. (It's a good idea to write a general purpose function you can reuse if you think you might have to do a specific task more than once or twice. This keeps you from having to copy and paste code too often, which saves time and can help prevent mistakes.)

In [93]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [match[0] for match in matches if match[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [95]:
# use the function we just wrote to replace close matches to "d.i khan" with "d.i khan"
replace_matches_in_column(df=df, column='City', string_to_match="d.i khan")

All done!


And now let's can check the unique values in our City column again and make sure we've tidied up d.i khan correctly.

In [92]:
# Get all the unique values in the 'City' column

City = df['City'].unique()

# sort them alphabetically and then take a closer look

City.sort()
City

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd.g khan', 'd.i khan', 'dara adam khel',
       'fateh jang', 'ghallanai, mohmand agency', 'gujrat', 'hangu',
       'haripur', 'hayatabad', 'islamabad', 'jacobabad', 'karachi',
       'karak', 'khanewal', 'khuzdar', 'khyber agency', 'kohat',
       'kuram agency', 'kurram agency', 'lahore', 'lakki marwat',
       'lasbela', 'lower dir', 'malakand', 'mansehra', 'mardan',
       'mohmand agency', 'mosal kor, mohmand agency', 'multan',
       'muzaffarabad', 'north waziristan', 'nowshehra', 'orakzai agency',
       'peshawar', 'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'],
      dtype=object)

Excellent! Now we only have "d.i khan" in our dataframe and we didn't have to change anything by hand.